In [1]:
# 동기 방식
import time


def main():
    for i in range(3):
        time.sleep(1)  # 블로킹
        print(i)


main()


0
1
2


In [6]:
# 노트북 전용 패턴
import asyncio


async def worker(i):
    await asyncio.sleep(1)  # 논블로킹
    print(i)


async def main():
    await asyncio.gather(*(worker(i) for i in range(3)))


# 여기! 노트북에서는 run() 대신 top-level await 사용
await main()  # 이미 돌아가는 이벤트 루프에 코루틴을 제출


0
1
2


In [10]:
import asyncio
import time


# ----------------------------
# (1) 동기 함수
# ----------------------------
def sync_task(name):
    print(f"[{name}] 시작")
    time.sleep(1)  # CPU를 점유하는 블로킹 동작
    print(f"[{name}] 끝")
    return f"{name} 완료"


# ----------------------------
# (2) 동기 함수를 async처럼 감싸기
# ----------------------------
async def async_wrap_sync_task(name):
    # run_in_executor는 동기 함수를 별도 스레드/프로세스에서 실행
    loop = asyncio.get_running_loop()
    return await loop.run_in_executor(None, sync_task, name)


# ----------------------------
# (3) 원래 async 함수 (논블로킹)
# ----------------------------
async def async_task(name):
    print(f"[{name}] 시작")
    await asyncio.sleep(1)  # 이벤트 루프에 제어권 반환
    print(f"[{name}] 끝")
    return f"{name} 완료"


# ----------------------------
# (4) 메인 실행
# ----------------------------
async def main():
    print("\n=== 동기 함수를 async처럼 감싼 경우 ===")
    start = time.perf_counter()
    results = await asyncio.gather(
        async_wrap_sync_task("동기-1"),
        async_wrap_sync_task("동기-2"),
        async_wrap_sync_task("동기-3"),
    )
    print("결과:", results)
    print("실행시간:", round(time.perf_counter() - start, 2), "초")

    print("\n=== 진짜 async 함수 ===")
    start = time.perf_counter()
    results = await asyncio.gather(
        async_task("비동기-1"),
        async_task("비동기-2"),
        async_task("비동기-3"),
    )
    print("결과:", results)
    print("실행시간:", round(time.perf_counter() - start, 2), "초")


# ----------------------------
# 실행 (노트북이면 await main(), 스크립트면 asyncio.run(main()))
# ----------------------------
if __name__ == "__main__":
    await main()



=== 동기 함수를 async처럼 감싼 경우 ===
결과: ['동기-1 완료', '동기-2 완료', '동기-3 완료']
실행시간: 1.0 초

=== 진짜 async 함수 ===
[비동기-1] 시작
[비동기-2] 시작
[비동기-3] 시작
[비동기-1] 끝
[비동기-2] 끝
[비동기-3] 끝
결과: ['비동기-1 완료', '비동기-2 완료', '비동기-3 완료']
실행시간: 1.01 초
